<a href="https://colab.research.google.com/github/Arjunerror/iooono/blob/main/Customer%20Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [ ]:
df.head().T

In [ ]:
df.totalcharges

In [9]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [10]:
df.totalcharges = df.totalcharges.fillna(0)

In [11]:
df.churn = (df.churn == 'yes').astype(int)

In [ ]:
df['churn']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
df_test.head()

In [ ]:
len(df_train), len(df_val), len(df_test)

In [17]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [18]:
df_full_train = df_full_train.reset_index(drop=True)

In [ ]:
df_full_train.isnull().sum()

In [ ]:
df_full_train.churn.value_counts(normalize=True)

In [ ]:
df_full_train.churn.mean()

In [23]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [24]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [ ]:
df_full_train[categorical].nunique()

In [ ]:
df_full_train.head()

In [ ]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

In [ ]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

In [ ]:
global_churn = df_full_train.churn.mean()
global_churn

In [ ]:
df_full_train.partner.value_counts()

In [ ]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

In [ ]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

In [ ]:
churn_no_partner / global_churn

In [ ]:
churn_partner / global_churn

SELECT gender, AVG(churn), AVG(churn) - global_churn AS diff, AVG(churn) / global_churn AS risk FROM data GROUP BY gender;

In [ ]:
from IPython.display import display
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)

In [36]:
from sklearn.metrics import mutual_info_score

In [ ]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

In [ ]:
mutual_info_score(df_full_train.gender, df_full_train.churn)

In [ ]:
mutual_info_score(df_full_train.contract, df_full_train.churn)

In [ ]:
mutual_info_score(df_full_train.partner, df_full_train.churn)

In [41]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [ ]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

In [ ]:
df_full_train[numerical].corrwith(df_full_train.churn)

In [ ]:
df_full_train[df_full_train.tenure >= 2].churn.mean()

In [ ]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()

In [ ]:
df_full_train[df_full_train.tenure > 12].churn.mean()

In [47]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
df_train[['gender','contract','tenure']].iloc[:10].to_dict(orient='records')

In [49]:
dicts = df_train[['gender','contract','tenure']].iloc[:100].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

In [ ]:
dv.fit(dicts)

In [ ]:
dv.get_feature_names_out()

In [ ]:
dv.transform(dicts)

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
print(type(train_dict))
X_train = dv.fit_transform(train_dict)
X_train

In [ ]:
dv.get_feature_names_out()

In [ ]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
X_val

In [56]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-7, 7, 51)
z

In [ ]:
sigmoid(10000)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(z, sigmoid(z))

In [61]:
def linear_regression(xi):
    result = w0

    for j in range(len(w)):
        result = result + xi[j] * w[j]

    return result

In [62]:
def logistic_regression(xi):
    score = w0

    for j in range(len(w)):
        score = score + xi[j] * w[j]

    result = sigmoid(score)
    return result

In [63]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

In [ ]:
model.intercept_[0]

In [ ]:
model.coef_[0]

In [ ]:
model.predict_proba(X_val)